<a href="https://colab.research.google.com/github/saeedsiddik/CMPUT501/blob/main/Notebook_To_Python_Conversion_using_manual_raw_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Package import

In [1]:
import os
import codecs
import json
import logging
import re

# Link to google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_directory = '/content/drive/MyDrive/Colab_Notebooks/data_selected_100/'
converted_data_directory = data_directory + '/converted_files'

In [ ]:
data_directory = 'data'
converted_data_directory = data_directory + '/converted_files'

# Function Filename Conversion 

In [4]:
def filename_conversion(filename):
    name = filename.replace('.ipynb', '.py')
    return name

# Input file selection Conversion 

In [5]:
def input_file_selection_function():
    raw_notebook_files = os.listdir(data_directory)

    selecetd_input_filenames = []
    selecetd_converted_filnames = []

    
    for filename in raw_notebook_files:        # Pick 20 notebooks for small execution
#         if filename.endswith('.ipynb') and filename.startswith('a'):
        if filename.endswith('.ipynb'):
            selecetd_input_filenames.append(filename)
            selecetd_converted_filnames.append(filename_conversion(filename))

    return selecetd_input_filenames, selecetd_converted_filnames

In [6]:
selecetd_input_filenames, selecetd_converted_filnames = input_file_selection_function()
print (len(selecetd_input_filenames))


100


In [12]:
print(selecetd_input_filenames[:10])

['zisu0426_a-begining-try.ipynb', 'kerneler_starter-msme-country-indicators-and-c73d2997-2.ipynb', 'jeffkk_guessing-the-objects-behind-the-classes.ipynb', 'ameerkhoso_indexing-selecting-assigning.ipynb', 'albou59_data-cleaning-challenge-handling-missing-v-3e8c4a.ipynb', 'tezdhar_map-ad-durations.ipynb', 'lini2390_video-game-sales-visualisation.ipynb', 'kalaychkan_evaluation-of-classification-algo-for-ham-spam.ipynb', 'canonwu_notebookbc0d2b7586.ipynb', 'hwmaianxun_eda-feature-engineering-and-everything-dca488.ipynb']


# Input file processing
The input ipynb files are processed individually. 

<!-- I have also checked the ML related packages and separate the file names based on their existence.  -->

I have also checked the bash script inside the code and commented those. 

Converted every single executable lines to Python3.6 code

In [7]:
import_package_list_with_file_name = []
import_package_list_with_file_name.append("FileName"+ "," + "Package" + "," + "Code" + "\n")

# Import package checking function

In [8]:
def import_package_checking(file_name, code_line_contains_import):
    code_token = code_line_contains_import.split(" ")
    
    if ((code_token[0] == 'import') or (code_token[0] == 'from')):
        
# pick the imported package info
        code_token_package = code_token[1]     

# However, there might be specific import using from keyword, which have a dot(.) between package and function
        import_package_token = code_token_package.split(".")[0]
    
        import_package_list_with_file_name.append(file_name + "," + import_package_token + "," + code_line_contains_import + "\n")
        

# write import package name in a csv file

In [9]:
def write_to_file_import_package_list_with_file_name(filename, import_package_list_with_file_name):
    try:
        write_to_file = open(filename, "w")
        write_to_file.flush()
        
        for content in import_package_list_with_file_name:
            write_to_file.write(content)
        
        write_to_file.close()
    except Exception as e:
        logging.error("Error Writing " + str(e) + "; in File: " + filename )

# Notebook processing cell by cell manully raw code

In [102]:
magic_functions_list = []
bash_scripts_list = []

def input_notebbok_processing_for_python_code(input_file_source_content, input_file_name):
    # ml_related_code_flag = False
    # ml_related_package_import = False

    json_content_of_file = json.loads(input_file_source_content)
    pySource = '### PythonConversion from ' + input_file_name + ' to ' + converted_file_name + ' ###\n'
    
    for file_content in json_content_of_file['cells']:

        try:
          if file_content['cell_type'] == 'code': 

              if isinstance(file_content['source'], str):            
                  code_lines = file_content['source'].split("\n")
                  for code_line in code_lines:
                    code_line_draft = code_line.strip()
                    if (code_line_draft.startswith('!')):
                      nonexecutable_code_line = '# ' + code_line_draft
                      bash_scripts_list.append([input_file_name, nonexecutable_code_line])
                      pySource = pySource + nonexecutable_code_line + "\n"
                      
                    elif (code_line_draft.startswith('%')):
                      nonexecutable_code_line = '# ' + code_line_draft
                      magic_functions_list.append([input_file_name, nonexecutable_code_line])
                      pySource = pySource + nonexecutable_code_line + "\n"
                    
                    else:
                      pySource = pySource + code_line + "\n"
              
              
              elif isinstance(file_content['source'], list):     

                  for code_line in file_content['source']:
                    code_line_draft = code_line.strip()
                    if (code_line_draft.startswith('!')):
                      nonexecutable_code_line = '# ' + code_line_draft
                      bash_scripts_list.append([input_file_name, nonexecutable_code_line])
                      pySource = pySource + nonexecutable_code_line + "\n"
                      
                    elif (code_line_draft.startswith('%')):
                      nonexecutable_code_line = '# ' + code_line_draft
                      magic_functions_list.append([input_file_name, nonexecutable_code_line])
                      pySource = pySource + nonexecutable_code_line + "\n"
                    
                    else:
                      pySource = pySource + code_line + "\n"

                              
          elif file_content['cell_type'] == 'markdown':
              for code_line in file_content['source']:
                  markdown_line_draft = code_line.strip()
                  markdown_line = '# ' + markdown_line_draft
                  pySource = pySource + markdown_line
                  if code_line[-1] != '\n':
                      pySource = pySource + '\n'
                  pySource = pySource + '\n'

        except Exception as e:
          logging.error("Error " + str(e) + "; in File: " + input_file_name )
          print(file_content['source'])

    # if (ml_related_code_flag):
    #     ml_related_notebooks_names.append(input_file_name)
    # else:
    #     traditional_notebooks_names.append(input_file_name)
    
    # if len(bash_scripts_list):
    #   print (bash_scripts_list)
    # if len(magic_functions_list):
    #   print (magic_functions_list)

    return pySource

# Write to converted files 

## write the distribution of non executable lines

In [114]:
write_to_nonexecutable_code_log(magic_functions_list, "magic_function_conversion_log.csv")

In [112]:
def write_to_nonexecutable_code_log(info_list, filename):
    try:
        write_to_file = open(converted_data_directory + filename, "w")
        for content in info_list:
          write_to_file.write(content[0] + "," + content[1] + "\n")
        write_to_file.close()
    except Exception as e:
        logging.error("Error Writing " + str(e) + "; in File: " + filename )

## Write to new .py file

In [12]:
def write_to_the_converted_file(pySource_content, converted_file_name):
    try:
        write_to_file = open(converted_data_directory + "/" + converted_file_name, "w")
        write_to_file.write(pySource_content)
        write_to_file.close()
    except Exception as e:
        logging.error("Error Writing " + str(e) + "; in File: " + converted_file_name )

In [13]:
def write_to_file(content, filename):
    try:
        write_to_file = open(filename, "w")
        write_to_file.write(content)
        write_to_file.close()
    except Exception as e:
        logging.error("Error Writing " + str(e) + "; in File: " + filename )

---
# Program Execution Main Part





In [ ]:

file_processing_counter_success = []
file_processing_counter_failure = []

# ml_related_list_initialize()

for input_file_name in selecetd_input_filenames:
    try:
        input_file = codecs.open(data_directory + "/" + input_file_name, 'r')

        converted_file_name = filename_conversion(input_file_name)

        input_file_source = input_file.read()
        python_content = input_notebbok_processing_for_python_code(input_file_source, input_file_name)

    #     print (python_content)   
        write_to_the_converted_file(python_content, converted_file_name)
        print (input_file_name + " : Complete")

        file_processing_counter_success.append(input_file_name)

    except Exception as e:
        logging.error("Error Reading " + str(e) + "; in File: " + input_file_name )
        file_processing_counter_failure.append(input_file_name)

print("Conversion Done; Total Sucess " + str(len(file_processing_counter_success)) + ", Fail " + str(len(file_processing_counter_failure)))

write_to_file_import_package_list_with_file_name("import_package_list1.csv", import_package_list_with_file_name)